In [24]:
from sage.coding.relative_finite_field_extension import *
index_arr=[i for i in range(1,6)]

def is_vulnerable_g2_cover(E, K, logs=False): 
    card=K.cardinality()
    if card^(1/3)!=floor(card^(1/3)):
        if logs:
            print("q!=p^3")
        return False
    else:
        if not is_prime(int(card^(1/3))):
            if logs:
                print("p=root(q,3) is not prime!")
            return False
    
    q=card
    X=polygen(K)    
    alpha=K.gen()
    if logs:
        print(E)
    
    k=K.degree()
    p=K.characteristic()
    
    irr = PolynomialRing(GF(p),'x').irreducible_element(2*k)
    K_2 = GF(q^2,'x',irr)
    t=RelativeFiniteFieldExtension(K_2,K)

    E_2=E.change_ring(t.embedding())
    
    Origin = E_2( 0 )
    #находим все элементы 2 кручения
    torsion_2=Origin.division_points( 2 )

    if logs:
        print(torsion_2)

    #фробениус, F(a)=a <=> a \in F_q^2
    frob_autom=q^2
    
    n=E.cardinality()
    if logs:
        print("No of points: ", n)
        
    tests_success=0; tests=0
    
    """
    for tmp in torsion_2:
        if tmp==torsion_2[0]:
            continue
        
        #считаем тесты
        tests+=1

        #проверяем лежат ли кручения в q^2 применяя соотв автоморф Фробениуса
        first=tmp[0]
        second=tmp[1]
        if logs:
            print(first, second)
            print(first^frob_autom,second^frob_autom)
        #если кручение в q^2, то ищем другое
        if first^frob_autom==first and second^frob_autom==second:
            if logs:
                print("Not proven yet...")
            tests_success+=1
            
        #иначе проверяем второе условие
        else:
            if logs:
                print("First condition failed")
            
            break
    
    if tests_success==tests:
        if logs:
            print("2-torsion in q^2 !")
        return(True)
    """
    if logs:
        print("Torsion", len(torsion_2), torsion_2)
    if 2^2 == len(torsion_2):
        return(True)
    
    if n%2==0:
        if logs:
            print("No! Not odd cardinality.")
        return(False)
        
    j_inv=E.j_invariant()
    #если j-инвариант неизменен при применении автоморфизма a->a^q, то он лежит в F_q
    if j_inv!=j_inv^(q):
        if logs:
            print("Yes! j-inv proof!")
        return(True)
        
    return(False)
        



def is_vulnerable_g3_cover(E, K, logs=False):    
    X = polygen(K)
    Kx=X.parent()
    fx=Kx(X^3)+Kx(E.a4())*Kx(X) + Kx(E.a6())
    
    num_pts=E.cardinality()
    if num_pts%4!=0:
        if logs:
            print("Divisable by 4 !")
        return(False)
        
    R=fx.roots()
    
    if logs:
        print(fx)
        print(R)
        
    for i in range(0,len(R)-1):
        for j in range(i+1,len(R)):
            #print(i,j)
            autom=p^3
            if R[i][0]^(autom)==R[j][0]:
                if logs:
                    print("Yes!", R[i][0], "|", R[j][0], "|", R[i][0]^(autom))
                    print("- - - ", autom)
                return(True)
            else:
                if logs:
                    print("No", R[i][0], "|",R[j][0], "|", R[i][0]^(autom))
    return(False)

def is_vulnerable_g9_cover(E, K, q, logs=False): 
    num_pts=E.cardinality()

    #проверяем число точек
    """
    if num_pts%4!=0:
        print("divisable by 4 !")
        return(False)
    print(num_pts)
    """
    J=E.j_invariant()
    if (J^(q^2)!=J and J^(q^3)!=J) and J^(q^6)==J:
        if logs:
            print("Continuing...")
    else:
        if logs:
            print((J^(q^2),J^(q^3),J^(q^6),J))
        return(False)
    
    X = polygen(K)
    Kx=X.parent()
    fx=Kx(X^3)+Kx(E.a4())*Kx(X) + Kx(E.a6())
    print(fx)
    
    #c4=E.b2()^2-24*E.b4()
    #print(c4^3/J, "_", E.discriminant())
    
    R=fx.roots()
    #проверяем, что sigma(a)=b для некого a, b
    
    for t in range(0,len(R)):
        tmp=roots[t]^q
        
        for k in range(0,len(R)):
            if t!=k:
                if logs:
                    print("Checking roots", R[t], "and", R[k], "_", tmp)
                if tmp==roots[k][0]:
                    if logs:
                        print("Yes!", roots[t],roots[k],tmp)
                    return(True)
        
    return(False)

def get_mod_poly(n, var_x, var_y):
    if n<1 or n>5:
        print("Not implemented!")
        return False
    
    filename="F_"+str(n)+".txt"
    #print("opening", filename)
    
    data = open(filename, "r").readlines()
    H = eval(",".join([s_.replace('\n','').replace(" ", ", ").replace("[","").replace("]","") for s_ in data]))
    
    #print(H)
    
    fxy=0; i=0
    while i<len(H):
        if H[i]!=H[i+1]:
            fxy+=var_x^(H[i])*var_y^(H[i+1])*H[i+2]+var_y^(H[i])*var_x^(H[i+1])*H[i+2]
        else:
            fxy+=var_x^(H[i])*var_y^(H[i+1])*H[i+2]
        
        #print(var_x^H[i], var_y^H[i+1])
        i+=3
    return fxy
    

def return_isogenies(E, K, num, logs=False):
    
    J=E.j_invariant()
    x, y = polygens(K, 'x, y')
    f=get_mod_poly(num,x,y)
    F=f(J,y)
    Y=polygen(K)
    Ky=Y.parent()

    roots_unextracted= [factor(F)[i][0] for i in range(0,len(factor(F)))]
    roots_=[roots_unextracted[i].coefficients() for i in range(0,len(roots_unextracted))]
    roots=[]
    for t in roots_:
        if len(t)==2:
            tmp=-t[1]
            roots.append(tmp)
    
    if logs:
        print("Все возможные J инварианты:", roots)
    ans=[]
    for root in roots:
        ans.append(EllipticCurve_from_j(root))
    
    return ans

def decode_voulnerability(code):
    out=""
    if code%2==1:
        out+="g2 voulnerable"
    code=code//2
    if code%2==1:
        out+=", g3 voulnerable"
    code=code//2
    if code%2==1:
        out+=", g9 voulnerable"
    if len(out)<1:
        return "Not voulnerable"
    return out

def check_voulnerability(E, K, logs=False):
    """
    возвращает три бита 000 - неуязвимая, 001 - уязвима к g2, ..., 111 - уязвима ко всем
    """
    flag=0
    
    if is_vulnerable_g2_cover(E, K, logs):
        if logs:
            print("g2 pass!")
        flag+=1
    if is_vulnerable_g3_cover(E, K, logs):
        if logs:
            print("g3 pass!")
        flag+=2
    if is_vulnerable_g9_cover(E, K, logs):
        if logs:
            print("g9 pass!")
        flag+=4
    return flag

def run_volkano(E, K, index_arr, logs=False):
    #возвращает массив изогенных кривых и массив уязвимых кривых
    if logs:
        print("Checking: ", E)
    isogen_curves=[]; vulnerable_curves=[]
    
    for index_ in index_arr:
        isogen_curves_tmp=return_isogenies(E, K, index_, logs)
        
        if logs:
            print("There are", len(isogen_curves),"curves...")
            print(isogen_curves_tmp)
            
        for instance in isogen_curves_tmp:
            #print(instance)   #debug
            tmp=check_voulnerability(instance, K, logs)
            if tmp:
                vulnerable_curves.append((instance,decode_voulnerability(tmp)))
            else:
                isogen_curves.append((instance))
            if logs:
                print("Curve: ", instance, decode_voulnerability(tmp))
    return isogen_curves, vulnerable_curves

def isogeny_walk(E_arr, K, current_index, max_index, ind_arr, done_arr, logs=False, upper_logs=True):
    #на входе чистим все повторения
    newlist=[]
    for i in E_arr: 
        if i not in done_arr: 
            done_arr.append(i) 
            newlist.append(i)
    E_arr=newlist
    
    #если рекурсия вернула превышение индекса, всплываем
    if current_index>=max_index:
        return "None."
    
    if upper_logs:
        print("Index:", current_index,"Indexes:" , ind_arr, "Checking:", len(E_arr),"curves.")
    
    next_E_arr=[]
    current_index+=1
    
    for t in E_arr:
        tmp=run_volkano(t, K, ind_arr, logs)
        #print(tmp)
        
        #если массив уязвмых кривых ненулевой, то мы нашли
        if len(tmp[1])>0:
            return tmp[1]
        #иначе добавляем все найденные изогенные кривые на след уровень поиска, редварительно удалив повторяющиеся
        for instance in tmp[0]:
            if not instance in E_arr:
                next_E_arr+=tmp[0]
    
    return isogeny_walk(next_E_arr, K, current_index, max_index, ind_arr, done_arr, logs)
    
    
            
#пример когда кривая уязвима ко 2 роду, но находится это только через 3 шага
p=5; k=3
K.<a> = GF(p^k)

done_arr=[]
index_arr=[1,2,3,4,5]
E=EllipticCurve(K,[2*a^2+a+3,a^2+1])
print(E)
print("First check: ",decode_voulnerability(check_voulnerability(E, K)))
#run_volkano(E, K, logs=True)
#run_volkano(E, K)
isogeny_walk([E], K, 0, 4, index_arr, done_arr, logs=False)

Elliptic Curve defined by y^2 = x^3 + (2*a^2+a+3)*x + (a^2+1) over Finite Field in a of size 5^3
First check:  Not voulnerable
Index: 0 Indexes: [1, 2, 3, 4, 5] Checking: 1 curves.
Index: 1 Indexes: [1, 2, 3, 4, 5] Checking: 3 curves.
Index: 2 Indexes: [1, 2, 3, 4, 5] Checking: 5 curves.


[(Elliptic Curve defined by y^2 = x^3 + (a^2+2)*x + (a^2+2*a) over Finite Field in a of size 5^3,
  'g2 voulnerable')]

In [25]:
p=7; k=6
K.<a> = GF(p^k)

E=EllipticCurve(K,[0,
                   4*a^5 + 6*a^3 + 5*a^2 + 5*a + 6,
                   0,
                   3*a^5 + 2*a^4 + 4*a^2 + 5*a + 6, 
                   3])
print("First check: ",decode_voulnerability(check_voulnerability(E, K)))
#run_volkano(E, K, logs=True)
#run_volkano(E, K)
const_max_polynomials=2
isogeny_walk([E], K, 0, 16, [1,2], [], logs=False)


First check:  Not voulnerable
Index: 0 Indexes: [1, 2] Checking: 1 curves.


[(Elliptic Curve defined by y^2 = x^3 + (a^5+4*a^4+3*a^3+2*a+4)*x + (5*a^5+a^4+2*a^2+6*a+2) over Finite Field in a of size 7^6,
  ', g3 voulnerable')]

In [26]:
isogeny_walk([E], K, 0, 20, [1,3], [], logs=False)

Index: 0 Indexes: [1, 3] Checking: 1 curves.


[(Elliptic Curve defined by y^2 = x^3 + (a^5+4*a^4+3*a^3+2*a+4)*x + (5*a^5+a^4+2*a^2+6*a+2) over Finite Field in a of size 7^6,
  ', g3 voulnerable')]

In [27]:
isogeny_walk([E], K, 0, 14, [1,4], [], logs=False)

Index: 0 Indexes: [1, 4] Checking: 1 curves.


[(Elliptic Curve defined by y^2 = x^3 + (a^5+4*a^4+3*a^3+2*a+4)*x + (5*a^5+a^4+2*a^2+6*a+2) over Finite Field in a of size 7^6,
  ', g3 voulnerable')]

In [28]:
isogeny_walk([E], K, 0, 8, [1,2,3], [], logs=False)

Index: 0 Indexes: [1, 2, 3] Checking: 1 curves.


[(Elliptic Curve defined by y^2 = x^3 + (a^5+4*a^4+3*a^3+2*a+4)*x + (5*a^5+a^4+2*a^2+6*a+2) over Finite Field in a of size 7^6,
  ', g3 voulnerable')]

In [30]:
p=7; k=6
K.<a> = GF(p^k)

E=EllipticCurve(K,[a^2+3,a^2+a+1])
print("First check: ",decode_voulnerability(check_voulnerability(E, K)))
#run_volkano(E, K, logs=True)
#run_volkano(E, K)
const_max_polynomials=2
isogeny_walk([E], K, 0, 50, [1,2,3], [], logs=False)


First check:  Not voulnerable
Index: 0 Indexes: [1, 2, 3] Checking: 1 curves.
Index: 1 Indexes: [1, 2, 3] Checking: 3 curves.
Index: 2 Indexes: [1, 2, 3] Checking: 6 curves.
Index: 3 Indexes: [1, 2, 3] Checking: 11 curves.
Index: 4 Indexes: [1, 2, 3] Checking: 22 curves.
Index: 5 Indexes: [1, 2, 3] Checking: 51 curves.


[(Elliptic Curve defined by y^2 = x^3 + (4*a^5+6*a^3+5*a^2+5*a+1)*x + (3*a^5+2*a^4+4*a^2+5*a+5) over Finite Field in a of size 7^6,
  ', g3 voulnerable')]

In [ ]:
def check_voulnerability_9(E, K, logs=False):
    """
    возвращает три бита 000 - неуязвимая, 001 - уязвима к g2, ..., 111 - уязвима ко всем
    """
    flag=0
    
    if is_vulnerable_g9_cover(E, K, logs):
        if logs:
            print("g9 pass!")
        flag+=4
    return flag

def run_volkano_9(E, K, index_arr, logs=False):
    #возвращает массив изогенных кривых и массив уязвимых кривых
    if logs:
        print("Checking: ", E)
    isogen_curves=[]; vulnerable_curves=[]
    
    for index_ in index_arr:
        isogen_curves_tmp=return_isogenies(E, K, index_, logs)
        
        if logs:
            print("There are", len(isogen_curves),"curves...")
            print(isogen_curves_tmp)
            
        for instance in isogen_curves_tmp:
            #print(instance)   #debug
            tmp=check_voulnerability_9(instance, K, logs)
            if tmp:
                vulnerable_curves.append((instance,decode_voulnerability(tmp)))
            else:
                isogen_curves.append((instance))
            if logs:
                print("Curve: ", instance, decode_voulnerability(tmp))
    return isogen_curves, vulnerable_curves

def isogeny_walk_9(E_arr, K, current_index, max_index, ind_arr, done_arr, logs=False, upper_logs=True):
    #на входе чистим все повторения
    newlist=[]
    for i in E_arr: 
        if i not in done_arr: 
            done_arr.append(i) 
            newlist.append(i)
    E_arr=newlist
    
    #если рекурсия вернула превышение индекса, всплываем
    if current_index>=max_index:
        return "None."
    
    if upper_logs:
        print("Index:", current_index,"Indexes:" , ind_arr, "Checking:", len(E_arr),"curves.")
    
    next_E_arr=[]
    current_index+=1
    
    for t in E_arr:
        tmp=run_volkano_9(t, K, ind_arr, logs)
        #print(tmp)
        
        #если массив уязвимых кривых ненулевой, то мы нашли
        if len(tmp[1])>0:
            return tmp[1]
        #иначе добавляем все найденные изогенные кривые на след уровень поиска, предварительно удалив повторяющиеся
        for instance in tmp[0]:
            if not instance in E_arr:
                next_E_arr+=tmp[0]
    
    return isogeny_walk_9(next_E_arr, K, current_index, max_index, ind_arr, done_arr, logs)
    
    